In [1]:
from pyspark.sql import SparkSession
import math

spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
sc = spark_session.sparkContext

In [2]:
# You can now use spark session or spark context to read csv.
train_data = spark_session.read.csv("/data/movielens/ratings_100k.csv",inferSchema=True,header=True)

In [3]:
from pyspark.mllib.recommendation import Rating
fold_count = 3

In [4]:
scores = []

rank = 5
numIterations = 5
lambda_ = 0.1

In [8]:
from pyspark.mllib.recommendation import ALS

a = 1-1/float(fold_count)
b = 1/float(fold_count)
for i in range(fold_count):

    '''alert, I performed sampling k-fold for this task, I can :
        1.Make random split t = train_data.randomSplit([1]*fold_count)
        2.on each iteration i=range(fold_count)
         2.1 test = t[iteration], 
         2.2 indices.remove(test_fold_index)
         2.3 train = sc.union([folds[i] for i in indices])
    '''
    train_fold, test_fold = train_data.randomSplit([a,b])
    ratings_traing = train_fold.rdd.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    ratings_test = test_fold.rdd.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
    evaluations = ratings_test.map(lambda l: (int(l[0]), int(l[1])))

    model = ALS.train(ratings_traing, rank, numIterations, lambda_= lambda_)
    
    
    predictions = model.predictAll(evaluations).map(lambda r: ((r[0], r[1]), r[2]))

    ratesAndPreds = ratings_test.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)


    MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
    scores.append(math.sqrt(MSE))

In [ ]:
# userID=26

# print("\nRatings for user ID " + str(userID) + ":")
# userRatings = ratings_train1.filter(lambda l: l[0] == userID)
# for rating in userRatings.collect():
#     print (rating[1],": ", str(rating[2]))

# print("\nTop 10 recommendations:")
# recommendations = model.recommendProducts(userID, 10)
# for recommendation in recommendations:
#     print (str(recommendation[1]) + " score " + str(recommendation[2]))


In [9]:
score = sum(scores) / len(scores)

In [10]:
print(score)

0.9392155583105842
